# Chroma DB

- 설치 패키지 목록: numpy pandas datasets chromadb sentence_transformers huggingface_hub[hf_xet] PyPDF2
- Python 3.9 버전이랑 호환성이 제일 좋음

In [ ]:
import chromadb

chroma_client = chromadb.Client()   # 임시 데이터베이스 클라이언트, 데이터를 메모리에만 저장함 → 세션 종료 시 모든 데이터가 사라짐

In [ ]:
collection = chroma_client.create_collection(name="my_collection")  # collection 객체 생성

In [ ]:
# 데이터 추가
collection.add(
    documents=[                     # 데이터 문장
        'This is a document about pineapple',
        'This is a document about mango',
        'This is a document about strawberry'
    ],
    ids=['id1', 'id2', 'id3']       # 문장을 구분할 id
)

/home/swhong/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:08<00:00, 10.1MiB/s]


In [ ]:
results = collection.query(
    query_texts=["This is a query document about vietnam"],     # 검색할 문자열
    n_results=1     # 유사도가 높은 문장 몇개를 찾을지
)

In [7]:
results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['This is a document about pineapple']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None]],
 'distances': [[1.2225853204727173]]}

### SciQ dataset 활용 ChromaDB 검색

In [ ]:
# SciQ 데이터셋 로드 : 과학적인 질문과 답변이 있는 데이터셋
from datasets import load_dataset

dataset = load_dataset("sciq", split="train")
dataset = dataset.filter(lambda x: x["support"] != "")  # support가 공백인 것 제외

dataset     # distractor: 보기(틀린답) | correct_answer: 정답 | support: 정답에 대한 설명

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 10481
})

In [1]:
# chroma db 클라이언트 객체 및 collection 생성
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="sciq_support")

In [ ]:
# 임베딩 모델 로드
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")   # 경량화된 임베딩 모델

/home/swhong/.conda/envs/vectordb_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
supports = dataset["support"][:100]     # 데이터가 너무 길어서 실습용으로 100개만 잘라서 사용
support_embeddings = embedding_model.encode(supports).tolist()

In [ ]:
len(support_embeddings[0])  # 384 차원으로 임베딩

384

In [8]:
collection.add(
    ids=[str(i) for i in range(0, 100)],
    embeddings=support_embeddings,
    metadatas=[{"type":"support", "text":text} for text in supports]
)

In [ ]:
questions = dataset["question"][:3]     # 질문 뽑아옴
question_embeddings = embedding_model.encode(questions).tolist()

results = collection.query(             # 검색
    query_embeddings=question_embeddings,
    n_results=1
)

In [14]:
results

{'ids': [['36'], ['1'], ['2']],
 'embeddings': None,
 'documents': [[None], [None], [None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'text': 'Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.',
    'type': 'support'}],
  [{'text': 'Without Coriolis Effect the global winds would blow north to south or south to north. But Coriolis makes them blow northeast to southwest or the re

In [13]:
for i, q in enumerate(questions):
    print("Questions:", q)
    print("Support:", results['metadatas'][i][0]['text'])
    print()

Questions: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.

Questions: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Support: Without Coriolis Effect the global winds would blow north to south or south to north

### Chroma DB를 활용한 키워드 기반 검색

In [15]:
documents = [
  "인공지능은 인간의 작업을 자동화하는 기술이다.",
  "기계 학습은 패턴을 학습하여 예측하는 기술이다.",
  "벡터 데이터베이스는 유사도를 기반으로 데이터를 검색하는 DB이다.",
  "자연어 처리는 인간의 언어를 컴퓨터가 이해하게 하는 기술이다.",
  "딥러닝은 신경망을 기반으로 한 기계 학습 기법이다.",
  "강화학습은 보상을 통해 최적의 행동을 학습하는 방식이다.",
  "컴퓨터 비전은 이미지나 영상을 분석하여 정보를 추출하는 기술이다.",
  "토큰화는 문장을 단어 또는 형태소 단위로 나누는 작업이다.",
  "임베딩은 텍스트나 데이터를 고정된 크기의 벡터로 변환하는 과정이다.",
  "LLM은 대규모 텍스트 데이터를 학습한 언어 모델이다."
]

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

# ChromaDB 클라이언트, 컬렉션 생성
client = chromadb.PersistentClient(path='./chroma_db')  # 영구 저장용 클라이언트, 데이터를 지정된 디렉토리에 영구 저장함 → 다시 실행해도 데이터 유지
collection = client.get_or_create_collection(name='ai_documents')   # .get_or_create_collection: 있으면 가져오고, 없으면 생성

# 텍스트 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

In [17]:
for i, doc in enumerate(documents):
    embedding = model.encode(doc).tolist()
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[{"text":doc}]
    )

In [21]:
query_keyword = '비전'

query_embedding = model.encode(query_keyword).tolist()

results = collection.query(query_embeddings=query_embedding, n_results=2)

results

{'ids': [['7', '3']],
 'embeddings': None,
 'documents': [[None, None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'text': '토큰화는 문장을 단어 또는 형태소 단위로 나누는 작업이다.'},
   {'text': '자연어 처리는 인간의 언어를 컴퓨터가 이해하게 하는 기술이다.'}]],
 'distances': [[0.9062471389770508, 1.0535051822662354]]}

In [22]:
for result in results["metadatas"][0]:
    print('검색된 문서:', result['text'])

검색된 문서: 토큰화는 문장을 단어 또는 형태소 단위로 나누는 작업이다.
검색된 문서: 자연어 처리는 인간의 언어를 컴퓨터가 이해하게 하는 기술이다.


### 영화 추천 시스템

- overview(줄거리) 임베딩 -> 저장
- 메타데이터 title 포함해서 저장
- 영화 제목을 입력 -> 유사한 영화 추천

- title 입력 받음 -> 입력받은 title로 overview 찾기 -> 단어 embedding -> vector db에서 검색 -> 추천 영화 반환

#### 내 방법

In [1]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

df = pd.read_csv('./data/tmdb_5000_movies.csv')
df = df[['overview', 'title']]
df = df.dropna().reset_index(drop=True)

overview_df = df['overview']
title_df = df['title']
title_search_df = df['title'].apply(lambda x: x.lower().replace(" ", ""))

# 텍스트 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')
chroma_client = chromadb.PersistentClient(path='./chroma_db')
collection = chroma_client.create_collection(name="movie_overview")  # collection 객체 생성


for i, doc in enumerate(overview_df):
    embedding_overview = model.encode(str(doc)).tolist()
    collection.add(
        ids=[f'id{i}'],
        embeddings=[embedding_overview],
        metadatas=[{"text":title_df[i]}]
    )

/home/swhong/.conda/envs/vectordb_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### 제목 입력해서 찾기

In [ ]:
input_ = input("영화 제목: ")
input_ = input_.lower().replace(" ", "")
query_text = overview_df[title_search_df==input_]
distance_threshold = 0.01


while len(query_text)==0: 
    input_ = input('없는 영화입니다. 다시 입력하세요: ')
    input_ = input_.lower().replace(" ", "")
    query_text = overview_df[title_search_df==input_]

results = collection.query(
    query_texts=query_text.iloc[0],     # 검색할 문자열
    n_results=5     # 유사도가 높은 문장 몇개를 찾을지
)

for distance, metadata in zip(results["distances"][0], results["metadatas"][0]):
    if distance > distance_threshold:
        print(f"추천 영화 {i}:", metadata["text"])

추천 영화: Alien: Resurrection
추천 영화: The Black Hole
추천 영화: Serenity
추천 영화: Aliens


##### 줄거리 입력해서 찾기

In [ ]:
query_text = input("영화 줄거리: ")

results = collection.query(
    query_texts=query_text,     # 검색할 문자열
    n_results=5     # 유사도가 높은 문장 몇개를 찾을지
)

for i, metadata in enumerate(results["metadatas"][0]):
    print(f"추천 영화 {i}:", metadata["text"])

추천 영화: Equilibrium
추천 영화: Thr3e
추천 영화: The Act of Killing
추천 영화: Contagion
추천 영화: River's Edge


##### 강사님 방법

### 논문 pdf 내용 검색

In [ ]:
# !pip install PyPDF2

In [1]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path='./chroma_db')
# client.delete_collection('papers')  # 컬렉션 삭제 -> 없으면 error 발생
collection = client.get_or_create_collection(name='papers')

model = SentenceTransformer('all-MiniLM-L6-v2')

In [2]:
papers = [
    {'id':'1', 'title':'내논문', 'path':'./data/my_paper.pdf'},
    {'id':'2', 'title':'NLP', 'path':'./data/nlp_paper.pdf'}
]

In [3]:
import PyPDF2

def extract_text_from_pdf(path):
    with open(path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        # .extract_text(): 페이지에서 텍스트 추출 | reader.pages: 페이지 추출
        text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()]) # 페이지에서 추출한 내용이 있을 때 공백으로 .join
    
    return text

In [4]:
for paper in papers:
    text = extract_text_from_pdf(paper['path']) # 텍스트 추출
    embedding = model.encode(text).tolist()     # embedding
    collection.add(
        ids=[paper['id']],
        embeddings=[embedding],
        metadatas=[{'title':paper['title']}],
        documents=[text]
    )

In [ ]:
collection.get()    # 저장된 전체 데이터

{'ids': ['1', '2'],
 'embeddings': None,
 'documents': ['Cement and Concrete Composites 152 (2024) 105646\nAvailable online 8 July 2024\n0958-9465/© 2024 The Author(s). Published by Elsevier Ltd. This is an open access article under the CC BY-NC license ( http://creativecommons.org/licenses/by-\nnc/4.0/ ).\nContents lists available at ScienceDirect\nCement and Concrete Composites\njournal homepage: www.elsevier.com/locate/cemconcomp\nMechanical property evaluation of 3D multi-phase cement paste\nmicrostructures reconstructed using generative adversarial networks\nSung-Wook Honga, Se-Yun Kima, Kyoungsoo Parka, Kenjiro Teradab, Hoonhee Leec,\nTong-Seok Hana,∗\naDepartment of Civil and Environmental Engineering, Yonsei University, Seoul 03722, Republic of Korea\nbInternational Research Institute of Disaster Science, Tohoku University, Sendai 980-8572, Japan\ncHalla Cement Corp., Gangneung, Gangwon 25645, Republic of Korea\nA R T I C L E I N F O\nKeywords:\nCement paste\nMicrostructure\nGe

In [7]:
results = collection.get(
    include=['embeddings', 'documents', 'metadatas']
)

for emb in results['embeddings']:
    print(len(emb))

384
384


In [11]:
query_text = 'Natural Language'
query_embedding = model.encode(query_text).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=1)

results['metadatas'][0][0]['title']

'NLP'